In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS etl_demo.gold;

-- Current view
CREATE OR REPLACE VIEW etl_demo.gold.customer_current AS
SELECT
  customer_id,
  name,
  city,
  valid_from
FROM etl_demo.silver.customer_dim
WHERE is_current = true;

-- KPI table
CREATE TABLE IF NOT EXISTS etl_demo.gold.customer_daily_kpi (
  kpi_date DATE,
  active_customers BIGINT,
  run_time TIMESTAMP
)
USING DELTA;

MERGE INTO etl_demo.gold.customer_daily_kpi tgt
USING (
  SELECT
    MAX(valid_from) AS kpi_date,
    COUNT(*) AS active_customers,
    current_timestamp() AS run_time
  FROM etl_demo.silver.customer_dim
  WHERE is_current = true
) src
ON tgt.kpi_date = src.kpi_date
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;
